In [1]:
import tensorflow as tf
from flask import Flask, request
import numpy as np
import mediapipe as mp
import cv2
import base64

In [2]:
model = tf.keras.models.load_model('my_model.keras')

In [3]:
actions = np.array(['hola', 'ok', 'a'])

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    right = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    left = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, right, left])

In [8]:
def decode_image(img):
    image_data = img.split(",")[1]
    image_bytes = base64.b64decode(image_data)
    image_buffer = np.frombuffer(image_bytes, dtype=np.uint8)
    frame = cv2.imdecode(image_buffer, cv2.IMREAD_COLOR)
    return frame

In [9]:
holistic =  mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [10]:
from flask import jsonify, render_template
from flask_socketio import SocketIO, send

from datetime import datetime


In [11]:
app = Flask(__name__)
socketio = SocketIO(app)
socketio.init_app(app, cors_allowed_origins="*")


frames = []
sequence = []
predictions = []
threshold = 0.5

@socketio.on("connect")
def on_connect():
    print("Cliente conectado")

@app.route("/")
def index():
    return render_template("index.html")

@socketio.on("message")
def on_message(data):   
   
    global sequence
    #print(data["frames"])
    image_base64 = data["frames"]
    #start_time = datetime.now()
    frame = decode_image(image_base64)
    #end_time = datetime.now()
    #print(end_time - start_time)
    
    image, results = mediapipe_detection(frame, holistic)
    keypoints = extract_keypoints(results)
    print(keypoints)
    sequence.append(keypoints)
    
    if len(sequence) == 30:
        #print(sequence[1])
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        prediction = actions[np.argmax(res)]
        socketio.emit("prediction", {"prediction": prediction})
        sequence = []

   
if __name__ == "__main__":
    socketio.run(app, host="localhost", port=5000, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2024 18:07:38] "GET /socket.io/?EIO=4&transport=polling&t=Owrgk16 HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2024 18:07:38] "POST /socket.io/?EIO=4&transport=polling&t=Owrgk61&sid=P6udfKLaMGo4qqb_AAAA HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2024 18:07:39] "GET /socket.io/?EIO=4&transport=polling&t=Owrgk61.0&sid=P6udfKLaMGo4qqb_AAAA HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2024 18:07:39] "GET /socket.io/?EIO=4&transport=polling&t=OwrgkAw&sid=P6udfKLaMGo4qqb_AAAA HTTP/1.1" 200 -


Cliente conectado
[ 0.62290782  0.33208644 -1.01544106 ...  0.          0.
  0.        ]
[ 0.62290782  0.33208644 -1.01544106 ...  0.          0.
  0.        ]
[ 0.62529188  0.33208954 -0.85898048 ...  0.          0.
  0.        ]
[ 0.62527931  0.33206898 -1.07074165 ...  0.          0.
  0.        ]
[ 0.62290215  0.33177361 -1.06918871 ...  0.          0.
  0.        ]
[ 0.6160745   0.32908732 -1.20763516 ...  0.          0.
  0.        ]
[ 0.58867824  0.32895839 -0.8295123  ...  0.          0.
  0.        ]
[ 0.54612315  0.31917059 -0.75945812 ...  0.          0.
  0.        ]
[ 0.52320826  0.29714265 -0.7814821  ...  0.          0.
  0.        ]
[ 0.51663846  0.29709771 -0.73549944 ...  0.          0.
  0.        ]
[ 0.50136107  0.30519426 -0.77613938 ...  0.          0.
  0.        ]
[ 0.49985409  0.32785824 -0.78890622 ...  0.          0.
  0.        ]
[ 0.5016073   0.32737529 -0.78331822 ...  0.          0.
  0.        ]
[ 0.50028425  0.32724619 -0.80917627 ...  0.          0.
  

[ 0.48278803  0.31159985 -1.28609407 ...  0.          0.
  0.        ]
[ 0.48212084  0.31207752 -1.34131718 ...  0.          0.
  0.        ]
[ 0.4821085   0.31343019 -1.3365562  ...  0.          0.
  0.        ]
[ 0.48199075  0.31442502 -1.31978476 ...  0.          0.
  0.        ]
[ 0.48180327  0.31574243 -1.31976306 ...  0.          0.
  0.        ]
[ 0.48178157  0.3162286  -1.28724694 ...  0.          0.
  0.        ]
[ 0.47979745  0.31677979 -1.28942931 ...  0.          0.
  0.        ]
[ 0.47921768  0.31693059 -1.2787261  ...  0.          0.
  0.        ]
[ 0.47897547  0.3174544  -1.28065622 ...  0.          0.
  0.        ]
[ 0.48272267  0.31864387 -1.25620472 ...  0.          0.
  0.        ]
[ 0.52095282  0.31772611 -1.16336274 ...  0.          0.
  0.        ]
[ 0.56298363  0.31429282 -1.11297822 ...  0.          0.
  0.        ]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[ 0.59318918  0.32139915 -1.08267558 ...  0.          0.
  0.        ]
[ 0.63825506  0.33199543 -0.96534288 ..

127.0.0.1 - - [06/Apr/2024 18:08:10] "GET /socket.io/?EIO=4&transport=websocket&sid=P6udfKLaMGo4qqb_AAAA HTTP/1.1" 200 -


[ 0.54662889  0.33195552 -1.00329041 ...  0.          0.
  0.        ]
